In [ ]:
import pandas as pd
from datetime import timedelta
import sys

In [ ]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance + 'Value'] = df[substance + 'Value'].astype(float)
    # finalTotal = finalTotal.drop(columns=['DateTime'])
    df = df.drop(columns=['datumBeginMeting'])
    # df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'])
    # df['StartDate'] = df['datumBeginMeting'].dt.date
    # df['StartTime'] = df['datumBeginMeting'].dt.time
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    # df['endDate'] = df['datumEindeMeting'].dt.date
    # df['endTime'] = df['datumEindeMeting'].dt.time
    # df = df.drop(columns=['datumBeginMeting','datumEindeMeting'])
    df.to_parquet('../data/cleanedData/'+ substance +'.parquet', index=False)
    pass

In [ ]:
data_files = [
    ('../data/Ammonium_measurements.parquet', 'ammonium'),
    ('../data/Nitrate_measurements.parquet', 'nitrate'),
    ('../data/Oxygen_A.parquet', 'oxygenA'),
    ('../data/Oxygen_B.parquet', 'oxygenB'),
    ('../data/Phosphate_measurements.parquet', 'phosphate')
]

for file_path, var_name in data_files:
    df = pd.read_parquet(file_path)
    cleanData(df, var_name)

In [ ]:
with open('../data/Total_influent_flow_WWTP_Ede_2021_minute_data.csv', 'r') as file:
    lines = file.readlines()

with open('../data/cleanedData/total.csv', 'w') as file:
    for line in lines:
        modified_line = line.replace(';60000', '').replace(';',',')
        file.write(modified_line)

finalTotal = pd.read_csv('../data/cleanedData/total.csv')
finalTotal['DateTime'] = pd.to_datetime(finalTotal['DateTime'], dayfirst=True)
# finalTotal['Date'] = finalTotal['DateTime'].dt.date
# finalTotal['Time'] = finalTotal['DateTime'].dt.time
# finalTotal = finalTotal.drop(columns=['DateTime'])
finalTotal.to_csv('../data/cleanedData/total.csv', index=False)

In [ ]:
ammonium = pd.read_parquet('../data/cleanedData/ammonium.parquet')
ammonium.head()

duplicates = ammonium[ammonium.duplicated(subset='datumEindeMeting', keep=False)].index.tolist()
# print(ammonium['datumEindeMeting'] == '2021-10-31 02:00:00')
march = ammonium.loc[ammonium['datumEindeMeting'] == '2021-03-28 03:00:00']
print(march.index)
print(duplicates)
ammonium.iloc[[436247,436248]]
#ammonium.iloc[[123849, 123850]]
#ammonium.iloc[[436366, 436367]]

In [ ]:
startTime = march.index.values[0]
endTime = duplicates[-1]

for i in range(startTime, endTime + 1):
    if i < (endTime - len(duplicates)):
        ammonium.loc[i, 'datumEindeMeting'] -= timedelta(hours=1)
    elif not ((i % 2) == 0):
        ammonium.loc[i, 'datumEindeMeting'] -= timedelta(hours=1)

duplicat = ammonium[ammonium.duplicated(subset='datumEindeMeting', keep=False)].index.tolist()
print(len(duplicat))
ammonium.iloc[[123849, 123850]]


In [ ]:
pd.set_option('display.max_rows', None)
print(duplicat)
print(len(duplicates))
# lol = ammonium.loc[ammonium['datumEindeMeting'] == '2021-10-31 02:00:00']
# print(lol)
#ammonium.iloc[123849:436368]
sorted_ammonium = ammonium.iloc[123849:436370].sort_values(by='datumEindeMeting').reset_index()
print(sorted_ammonium)
#ammonium = ammonium.sort_values(by='datumEindeMeting')
#ammonium[436367]
